# Loading and Preparing Data for Training
**Import Libraries**

**Load the JSON data**

**Create lists to hold the patterns and corresponding tags**

**Extract patterns and tags from the JSON data**

**Create a DataFrame from the lists**

**Split the data into training and validation sets**

**Save the training and validation sets to CSV files**

In [ ]:
import json
import torch
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load the JSON data
with open('/kaggle/input/mental-health-bot/KB.json', 'r') as file:
    data = json.load(file)

patterns = []
tags = []
responses = {}

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses[intent['tag']] = ['responses']


df = pd.DataFrame({
    'text': patterns,
    'label': tags
})

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

# **Tokenization, Dataset Preparation, and Model Training with BERT**
**Load Tokenizer and Tokenize Text**

**Create Label Mappings**

To convert categorical labels (text) to numerical values and vice versa, facilitating training and evaluation.

**Create a Custom Dataset Class**

To organize the data in a format compatible with PyTorch, enabling efficient data loading and batching during training.

**Create Dataset Instances**

Creating instances of the custom dataset for training and validation ensures the data is correctly formatted and accessible for the training process

**Set Training Arguments**

**Initialize the Trainer and Start Training**

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True)

label2id = {label: idx for idx, label in enumerate(df['label'].unique())}
id2label = {idx: label for label, idx in label2id.items()}
train_labels = train_df['label'].map(label2id).tolist()
val_labels = val_df['label'].map(label2id).tolist()

class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IntentDataset(train_encodings, train_labels)
val_dataset = IntentDataset(val_encodings, val_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label2id))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

2024-06-21 11:02:21.895121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 11:02:21.895282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 11:02:22.084867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to q

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.544400,0.491771
2,0.081600,0.233004
3,0.113700,0.204191


TrainOutput(global_step=1878, training_loss=0.7600456657834327, metrics={'train_runtime': 4652.5953, 'train_samples_per_second': 3.227, 'train_steps_per_second': 0.404, 'total_flos': 254756735114112.0, 'train_loss': 0.7600456657834327, 'epoch': 3.0})

# **Save the model and tokenizer**

In [3]:
model.save_pretrained('bert-mental-health-model')
tokenizer.save_pretrained('bert-mental-health-tokenizer')

('bert-mental-health-tokenizer/tokenizer_config.json',
 'bert-mental-health-tokenizer/special_tokens_map.json',
 'bert-mental-health-tokenizer/vocab.txt',
 'bert-mental-health-tokenizer/added_tokens.json')

# **Evaluating the result**

In [4]:
eval_result = trainer.evaluate()

print(f"Evaluation results: {eval_result}")

Evaluation results: {'eval_loss': 0.20419113337993622, 'eval_runtime': 60.4304, 'eval_samples_per_second': 20.701, 'eval_steps_per_second': 1.307, 'epoch': 3.0}


# **Predicting the Intent of a random text with the trained model and tokenizer**

In [5]:
model = BertForSequenceClassification.from_pretrained('bert-mental-health-model')
tokenizer = BertTokenizer.from_pretrained('bert-mental-health-tokenizer')

def predict_intent(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = predictions.argmax().item()
    return id2label[predicted_class]

text = "Hi, I need help with my anxiety."
intent = predict_intent(text)
print(f"Predicted intent: {intent}")

Predicted intent: anxious


# **Predicting the response of a text**

In [7]:
model = BertForSequenceClassification.from_pretrained('/kaggle/working/bert-mental-health-model')
tokenizer = BertTokenizer.from_pretrained('/kaggle/working/bert-mental-health-tokenizer')

def predict_intent_and_respond(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = predictions.argmax().item()
    intent = id2label[predicted_class]
    
    response = random.choice(responses[intent])
    return response

text = "Hi, I need help with my anxiety."
response = predict_intent_and_respond(text)
print(f"Response: {response}")

Response: responses
